In [6]:
import pandas as pd
import matplotlib.pyplot as plt

In [32]:
df = pd.read_csv('usable_sp2019.csv', index_col=0)

In [11]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
model = RandomForestClassifier(n_estimators=1000, min_samples_split=50, random_state=1)

In [12]:
from sklearn.metrics import precision_score

In [36]:
def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Target"])
    preds = model.predict(test[predictors])
    preds = pd.Series(preds, index=test.index)
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [26]:
train = df.sample(frac=.8)
test = df.drop(train.index)
train = train.reset_index()
test = test.reset_index()
del train['index']
del test['index']


In [33]:
# predictors
predictors = ["Close", "Open", "High", "Low", "Prev 1", "Prev 2","Prev 3","Prev 4","Prev 5","Prev 6","Prev 7",
              "Prev 8","Prev 9","Prev 10"]

In [35]:
# backtest by looking at how the vid used only the testing data to return the concat
def backtest(model, predictors, train, test):
    model.fit(train[predictors], train["Target"])
    preds = predict(train, test, predictors, model)
    combined = pd.concat([test["Target"], preds], axis=1)
    return combined

In [37]:
predictions = backtest(model, predictors, train, test)

In [60]:
precision_score(predictions["Target"], predictions["Predictions"])

0.6198582956111002

In [61]:
predictions["Target"].value_counts() / predictions.shape[0]

1    0.510372
0    0.489628
Name: Target, dtype: float64

In [62]:
predictions[100:140]

,Target,Predictions
100,1,0
101,1,0
102,0,0
103,1,0
104,1,0
105,1,0
106,1,0
107,1,1
108,1,1
109,0,0


In [63]:
import pickle

In [64]:
filename = 'base_tree.sav'
pickle.dump(model, open(filename, 'wb'))